# from brain import understanding

In [1]:


def getAdj(a, p, prox = 0):
    """
    Find all adjacent and accurate boxes for each box in a list of boxes.
    :param a: a list of boxes
    :param p: p list of boxes
    :param prox: proximity threshold for adjacency
    :return: a, p updated lists of boxes with adjacency and accuracy information
    """
    for box in p.values():
        for otherbox in a.values():
            if box['right'] > otherbox['left'] and box['left'] < otherbox['right'] and box['boxID'] not in otherbox['acc']:
                box['acc'].append(otherbox['boxID'])
                otherbox['acc'].append(box['boxID'])
                
        for otherbox in p.values():
            if box['right'] + prox > otherbox['left'] and box['left'] < otherbox['right'] + prox and box['boxID'] not in otherbox['adj']:
                box['adj'].append(otherbox['boxID'])
                otherbox['adj'].append(box['boxID'])
                
    return a, p


In [2]:


def read_boxes(boxestxt, predict = True):
    """
    Read the specified text file and turn it into a list of dictionaries, where each 
    dictionary represents a predicted box.
    
    Input: 
        boxestxt: string name of text file in working directory with lines representing boxes 
           from either hand annotations or model output
        predict: boolean True if boxes are predictions, false if annotations
    
    Output: 
        predictions / annotations: list of dictionaries representing boxes from the text file, sorted 
            by box ID (top-down order in text file)
    
    """
    
    # read text file line by line
    text = open(boxestxt, 'r')
    lines = [line.split(sep = "\t") for line in text.readlines()]
    headers = lines.pop(0)
    predictions = {}
    annotations = {}
    boxID = 0
    
    
    if predict == True:
    
        # create box for each line where (left, top) is top left corner and (right, bottom) is bottom right corner
        for box in lines:
            predictions[boxID] = {
                 "boxID":boxID, 
                 "left":float(box[0]), 
                 "top":float(box[3]), 
                 "right":float(box[1]), 
                 "bottom":float(box[2]), 
                 "conf":float(box[4]), 
                 "class":None, 
                 "adj":[], 
                 "acc":[],
                 "dur":float(box[1])-float(box[0])
            }
            boxID += 1

        return predictions
    
    elif predict == False:
        
        # create box for each line where (left, top) is top left corner and (right, bottom) is bottom right corner
        for box in lines:
            annotations[boxID] = {
                 "boxID":boxID, 
                 "left":float(box[3]), 
                 "top":float(box[6]), 
                 "right":float(box[4]), 
                 "bottom":float(box[5]), 
                 "class":None, 
                 "adj":[],
                 "acc":[],
                 "dur":float(box[4])-float(box[3])
            }
            boxID += 1
        
        return annotations
    

    
    return None


In [3]:


def get_iou(box1, box2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Input: 
        box1, box2: Two dictionaries representing boxes (must have keys left, right, top, bottom) where 
        (left, top) is top left corner and (right, bottom) is bottom right corner

    Output: 
        float on [0,1] representing the Intersection over Union
    
    """
    
    try:
        assert box1['left'] <= box1['right']
        assert box1['top'] >= box1['bottom']
        assert box2['left'] <= box2['right']
        assert box2['top'] >= box2['bottom']
    except(AssertionError):
        raise AssertionError("Invalid bounding box")

    # determine the coordinates of the intersection rectangle
    left = max(box1['left'], box2['left'])     # larger of the two left bounds
    bottom = max(box1['bottom'], box2['bottom'])   # larger of the two bottom bounds
    right = min(box1['right'], box2['right'])    # smaller of the two right bounds
    top = min(box1['top'], box2['top'])      # smaller of the two upper bounds

    if right < left or top < bottom:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (right - left) * (top - bottom)

    # compute the area of both AABBs
    box1_area = (box1['right'] - box1['left']) * (box1['top'] - box1['bottom'])
    box2_area = (box2['right'] - box2['left']) * (box2['top'] - box2['bottom'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    #print(box1, box2)
    try:
        iou = intersection_area / float(box1_area + box2_area - intersection_area)
    except ZeroDivisionError:
        print("Two bounding boxes with a nonpositive dimension found. Treating IoU as 0.")
        iou = 0
    assert iou >= 0.0
    assert iou <= 1.0
    return iou



In [4]:


def getOverlapCoef(box1, box2):
    """
    Calculate the overlap coefficient of two bounding boxes.

    Input: 
        box1, box2: Two dictionaries representing boxes (must have keys x1, x2, y1, y2) where 
        (left, top) is top left corner and (right, bottom) is bottom right corner

    Output: 
        float on [0,1] representing the overlap coefficient
    
    """
    
    try:
        assert box1['left'] <= box1['right']
        assert box1['top'] >= box1['bottom']
        assert box2['left'] <= box2['right']
        assert box2['top'] >= box2['bottom']
    except(AssertionError):
        raise AssertionError("Invalid bounding box")

    # determine the coordinates of the intersection rectangle
    left = max(box1['left'], box2['left'])     # larger of the two left bounds
    bottom = max(box1['bottom'], box2['bottom'])   # larger of the two bottom bounds
    right = min(box1['right'], box2['right'])    # smaller of the two right bounds
    top = min(box1['top'], box2['top'])      # smaller of the two upper bounds

    if right < left or top < bottom:
        return 0.0
    
    # determine the area of overlap
    intersection_area = (right - left) * (top - bottom)
    
    # return ratio of overlap area to area of smaller of the two boxes
    a_box1 = (box1['right'] - box1['left']) * (box1['top'] - box1['bottom'])
    a_box2 = (box2['right'] - box2['left']) * (box2['top'] - box2['bottom'])
    
    ovr = intersection_area / min(a_box1, a_box2)
    
    if ovr > 1:
        raise ValueError("Invalid overlap")
    
    return ovr



In [5]:


def nms(predicted, threshold = None, ovr = 'ovr'):
    """
    
    NON MAXIMUM SUPPRESSION
    
    Filters boxes so that only high-confidence boxes with lower-than-threshold overlap remain for the same class
    
    
    Input: 
        predicted: list of predicted boxes
        threshold: overlap metric above which boxes will be suppressed
        ovr: overlap method (iou for intersection over union or ovr for overlap coefficient)
        
    Output: 
        filtered: list of predicted boxes, filtered to maximize high confidence and low overlap
    
    """
    
    # NMS Threshold is set to None, so we want no filtering
    if threshold is None:
        return predicted
    
    # Implements NMS Algorithm on whole predicted list
    b = sorted(predicted.values(), key = lambda d: d['conf'], reverse = True)
    print(b)
    checking = b.pop(0)
    d = []
    while b:
        if checking['adj']:
            for box in b:
                if getOverlapCoef(checking, box) > threshold:
                    b.remove(box)
        checking = b.pop(0)
        d.insert(0, checking)
                
    return {box['boxID']:box for box in d}


    '''

    a good explaination of NMS and its algorithm:
    https://towardsdatascience.com/non-maximum-suppression-nms-93ce178e177c

    '''


In [6]:


def getBoxSuccess(predicted, annotated, thresh = 0.8, compare = "iou"):
    """
    Calculates the number of true positives, false positives, and false negatives for a list of predictions and a list of hand-annotations.
    
    Input:
        predicted: list of dictionaries representing predicted boxes
            
        annotated: list of dictionaries representing ground truth labels
            
        thresh: float [0,1]
        
        compare: method ["iou", "ovr"] for comparing boxes
    
    Output:
    
        metrics: dictionary with model metrics
        
        
        attributes/metrics:
    
        truePositives: int
            The number of predicted boxes in "predicted" that have enough overlap with a hand-annotated box in "annotated"
            (Note that two predicted boxes can have enough overlap with the same hand-annotated box, and this counts as two true positives.)
            
        falsePositives: int
            The number of predicted boxes in "predicted" that do not have enough overlap with a hand-annotated box in 
                "annotated".
                
        falseNegatives: int
            The number of label boxes in "annotated" that do not have enough overlap with a predicted box in 
                "predicted".
                
        numPredicted: int representing predicted box count
            
        numAnnotated: int representing ground truth box count
        
        nonbinary accuracy: float representing proportion of annotations captured by a box. - hopefully will work soon

    """
    
    
    
    if annotated is None or predicted is None:
        print("Input Missing")
        return None
    
    
    truePositives = 0
    falsePositives = 0
    falseNegatives = 0
    
    numPredicted = len(predicted)
    numAnnotated = len(annotated)
    
    
    
    # find adjacency between predictions and annotations
    annotated, predicted = getAdj(annotated, predicted)
    
    # count positives and negatives
    for prediction in predicted.values():
        fp = True
        for box in prediction['acc']:
            if compare == "iou" and get_iou(prediction, annotated[box]) > thresh:
                fp = False
                break
            elif compare == "ovr" and getOverlapCoef(prediction, annotated[box]) > thresh:
                fp = False
                break
        if fp:
            falsePositives += 1
            
    for annotation in annotated.values():  
                
        fn = True
        for box in annotation["acc"]:
            if not box in predicted.keys():
                continue
            if compare == "iou" and get_iou(annotation, predicted[box]) > thresh:
                fn = False
                truePositives += 1
                break
            elif compare == "ovr" and getOverlapCoef(annotation, predicted[box]) > thresh:
                fn = False
                truePositives += 1
                break
        if fn:
            falseNegatives +=1
    
    # find nonbinary accuracy (average overlap per prediction)
    accs = []
    for prediction in predicted.values():
        x = prediction["conf"]
        try:
            weight = -((1 - (x - 1)**4)/(1 - (x - 1)**2)) + 2
        except(ZeroDivisionError):
            weight = 0
        if compare == "iou":
            if prediction["acc"]:
                accs.append(get_iou(prediction, annotated[prediction["acc"][0]]) * weight)
            continue
        elif compare == "ovr":
            if prediction["acc"]:
                accs.append(getOverlapCoef(prediction, annotated[prediction["acc"][0]]) * weight)
            continue
        else:
            print("Unsupported overlap comparison method")
            return None
        
    binaryPrecision = truePositives/(truePositives + falsePositives)
    binaryAccuracy = truePositives/numPredicted
    binaryRecall = truePositives/(truePositives + falseNegatives)

    return {"numPredicted":numPredicted, 
            "numAnnotated":numAnnotated,
        
            "truePositives":truePositives, 
            "falsePositives":falsePositives, 
            "falseNegatives":falseNegatives, 
            
            "binaryAccuracy":binaryAccuracy,
            "binaryPrecision":binaryPrecision, 
            "BinaryRecall":binaryRecall,
            "binaryF1":2*binaryPrecision*binaryRecall/(binaryPrecision+binaryRecall) if binaryPrecision+binaryRecall != 0 else 0.0,
            
            "nonbinaryAccuracy":sum(accs)/len(accs) if len(accs) != 0 else 0.0}    
    
      

In [7]:


def filterBoxes(predicted, dim = 'conf', upper = True, thresh = 0.1):
    
    output = {}
    for box in predicted.keys():
        if upper and predicted[box][dim] > thresh:
            output[box] = predicted[box]
        if not upper and predicted[box][dim] < thresh:
            output[box] = predicted[box]
                
    return output
    
    

## BOX COMBINATION

In [8]:

### CURRENTLY DOES NOT WORK ###


def combine(predicted, cluster, i):
    
    boxes = [predicted[n] for n in cluster]
    
    left = min([box['left'] for box in boxes])
    right = max([box['right'] for box in boxes])
    top = max([box['top'] for box in boxes])
    bottom = min([box['bottom'] for box in boxes])
    
    combined = {"boxID":i, "class":boxes[0]["class"], "left":left, "top":top, "right":right, "bottom":bottom, "conf":max([box["conf"] for box in boxes]), "adj":[], "acc":[]}
    
    return combined


def combine_overlaps(predicted, prox = 0):
    """    
    Combines all clusters of boxes into single boxes and keeps maximum confidence level and minimum boxID
    
    Input:
        predicted: list of dictionaries
    
    Output:
        filt_predicted: list of lists contains predicted boxes (sorted by confidence score) after applying the overlap threshold

    """
    
    predicted = getAdj(predicted, predicted, prox)[1]
    
    cliques = [[box['boxID']] + box['acc'] for box in predicted.values()]
    
    combined = {}
    i = 0
    for cluster in cliques:
        combined[i] = combine(predicted, cluster, i)
        i+=1
    
    return combined


# Code to evaluate how well our model does to the human annotations

In [ ]:
# ground = read_boxes('671658014.180928183606_annotations.txt', False)
# output = read_boxes('671658014.180928183606_predictions.txt', True)

In [ ]:
# fOut = nms(
#     filterBoxes(
#         filterBoxes(
#             filterBoxes(
#                 output,
#                 dim = 'top',
#                 upper = False,
#                 thresh = 2400
#             ), 
#             thresh = 0.3
#         ),
#         dim = 'dur',
#         thresh = 0.25
        
#     ), 
#     0.5
# )

In [ ]:
# # raw model output metrics

# getBoxSuccess(
#     predicted = fOut,
#     annotated = ground,
#     thresh = 0.6,
#     compare = "ovr"
# )

In [ ]:
# sorted(ground.values(), key = lambda d: d['top'], reverse = True)